# Built-in Middlewares for HR Agents - LangChain 1.0

**Module:** Built-in Middleware Components

**What you'll learn:**
- 📝 SummarizationMiddleware - Automatic conversation summarization
- 👤 HumanInTheLoopMiddleware - Manual approval workflows
- 🎯 Production patterns for HR use cases

**HR Use Cases:**
- Long employee consultation sessions with memory management
- Critical HR decisions requiring manager approval
- Salary updates with multi-level authorization
- Compliance-driven approval workflows

**Time:** 1-2 hours

---

## Setup: Install Dependencies

In [ ]:
# Install LangChain 1.0 and middleware packages
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite

## Setup: Configuration and Imports

In [ ]:
# Configure API key
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Imports
from langchain.agents import create_agent, AgentState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from datetime import datetime
import json

print("✅ Setup complete!")

## Setup: HR Tools and Data

In [ ]:
# HR Employee Database
EMPLOYEES = {
    "101": {
        "name": "Priya Sharma",
        "department": "Engineering",
        "role": "Senior Developer",
        "salary": 120000,
        "leave_balance": 12,
        "manager_id": "102"
    },
    "102": {
        "name": "Rahul Verma",
        "department": "Engineering",
        "role": "Engineering Manager",
        "salary": 180000,
        "leave_balance": 8,
        "manager_id": "103"
    },
    "103": {
        "name": "Anjali Patel",
        "department": "HR",
        "role": "HR Director",
        "salary": 200000,
        "leave_balance": 15,
        "manager_id": None
    },
    "104": {
        "name": "Arjun Reddy",
        "department": "Sales",
        "role": "Sales Team Lead",
        "salary": 150000,
        "leave_balance": 10,
        "manager_id": "105"
    },
    "105": {
        "name": "Sneha Gupta",
        "department": "Sales",
        "role": "Sales Director",
        "salary": 190000,
        "leave_balance": 5,
        "manager_id": "103"
    }
}

# Define HR tools
@tool
def get_employee_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get employee information by ID."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} - {emp['department']} - {emp['role']}"
    return f"Employee {employee_id} not found"

@tool
def check_leave_balance(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Check leave balance for an employee."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} has {emp['leave_balance']} days of leave remaining"
    return f"Employee {employee_id} not found"

@tool
def get_salary_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get salary information. SENSITIVE operation."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']}'s annual salary: ₹{emp['salary']:,}"
    return f"Employee {employee_id} not found"

@tool
def update_salary(
    employee_id: Annotated[str, "Employee ID"],
    new_salary: Annotated[int, "New salary amount"]
) -> str:
    """Update employee salary. CRITICAL operation requiring approval."""
    if employee_id in EMPLOYEES:
        old_salary = EMPLOYEES[employee_id]['salary']
        EMPLOYEES[employee_id]['salary'] = new_salary
        return f"✅ Salary updated for {EMPLOYEES[employee_id]['name']}: ₹{old_salary:,} → ₹{new_salary:,}"
    return f"Employee {employee_id} not found"

@tool
def approve_leave(
    employee_id: Annotated[str, "Employee ID"],
    days: Annotated[int, "Number of leave days"]
) -> str:
    """Approve leave request. Requires manager approval."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        if emp['leave_balance'] >= days:
            EMPLOYEES[employee_id]['leave_balance'] -= days
            return f"✅ Approved {days} days leave for {emp['name']}. Remaining: {EMPLOYEES[employee_id]['leave_balance']} days"
        return f"❌ Insufficient leave balance. {emp['name']} has only {emp['leave_balance']} days"
    return f"Employee {employee_id} not found"

print("✅ HR tools and data configured!")
print(f"Total employees: {len(EMPLOYEES)}")

---
# Part 1: SummarizationMiddleware 📝

## Why Summarization?

**Problem:** Long HR consultation sessions exceed LLM context windows

**Example Scenario:**
```
Employee: "I joined in 2020..."
HR Agent: "Great! Tell me more..."
Employee: "I work in Engineering..."
HR Agent: "What can I help with?"
Employee: "Need leave for wedding..."
...
[After 50 messages, context is too long!]
```

**Solution:** SummarizationMiddleware automatically:
- Monitors message token count
- Summarizes old messages when threshold reached
- Keeps recent messages intact
- Preserves conversation context

---

## Lab 1.1: Basic Summarization Setup

In [ ]:
# Note: As of LangChain 1.0, SummarizationMiddleware is available via:
# from langchain.agents import SummarizationMiddleware

# For this demo, we'll implement a simplified version
# that demonstrates the concept

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

class SimpleSummarizationMiddleware:
    """Simplified summarization middleware for demonstration."""
    
    def __init__(
        self,
        model: ChatOpenAI,
        max_tokens: int = 2000,
        messages_to_keep: int = 5
    ):
        self.model = model
        self.max_tokens = max_tokens
        self.messages_to_keep = messages_to_keep
        self.summary = None
    
    def estimate_tokens(self, messages) -> int:
        """Rough token estimation."""
        total = 0
        for msg in messages:
            if hasattr(msg, 'content'):
                # Rough estimate: ~1.3 tokens per word
                total += len(msg.content.split()) * 1.3
        return int(total)
    
    def summarize_messages(self, messages) -> str:
        """Create summary of old messages."""
        # Prepare conversation for summarization
        conversation = "\n".join([
            f"{msg.type}: {msg.content}" 
            for msg in messages 
            if hasattr(msg, 'content')
        ])
        
        summary_prompt = f"""Summarize this HR conversation concisely:

{conversation}

Summary (2-3 sentences):"""
        
        response = self.model.invoke([HumanMessage(content=summary_prompt)])
        return response.content
    
    def pre_model_hook(self, state: AgentState) -> dict:
        """Check if summarization is needed."""
        messages = state.get("messages", [])
        
        if len(messages) < self.messages_to_keep:
            return {}
        
        # Estimate tokens
        token_count = self.estimate_tokens(messages)
        
        print(f"\n📊 Token count: ~{token_count} (threshold: {self.max_tokens})")
        
        if token_count > self.max_tokens:
            print(f"🔄 Summarizing old messages...")
            
            # Messages to summarize (all except recent)
            to_summarize = messages[:-self.messages_to_keep]
            recent_messages = messages[-self.messages_to_keep:]
            
            # Create summary
            summary = self.summarize_messages(to_summarize)
            self.summary = summary
            
            print(f"✅ Summary created: {len(to_summarize)} messages → {len(summary.split())} words")
            
            # Replace old messages with summary
            summary_message = SystemMessage(content=f"**Previous conversation summary:**\n{summary}")
            new_messages = [summary_message] + recent_messages
            
            return {"messages": new_messages}
        
        return {}

print("✅ SimpleSummarizationMiddleware created!")

## Lab 1.2: HR Consultation with Summarization

In [ ]:
# Create summarization middleware
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summarization_mw = SimpleSummarizationMiddleware(
    model=llm,
    max_tokens=500,  # Low threshold for demo
    messages_to_keep=3
)

# Create agent with summarization
hr_consultation_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, get_salary_info],
    pre_model_hook=summarization_mw.pre_model_hook,
    checkpointer=InMemorySaver(),
    prompt="""You are a helpful HR consultant.
    
    Help employees with:
    - General information
    - Leave balance inquiries
    - Career guidance
    - Policy questions
    
    Be friendly, professional, and remember conversation context."""
)

print("✅ HR Consultation Agent with Summarization ready!")

## Lab 1.3: Test Long Conversation with Auto-Summarization

In [ ]:
config = {"configurable": {"thread_id": "consultation_session_1"}}

# Simulate long consultation session
consultation_messages = [
    "Hi, I'm Priya Sharma, employee 101. I have some questions.",
    "I joined the company in 2020 and I work in the Engineering department.",
    "Can you tell me about my current role and responsibilities?",
    "I'm interested in understanding the career progression path for senior developers.",
    "What are the typical skills needed to become an Engineering Manager?",
    "How many days of leave do I have remaining this year?",
    "I'm planning to take a vacation next month. What's the leave approval process?",
    "Can you check my salary information?",
    "What benefits am I eligible for at my current level?",
    "Tell me about the company's professional development programs."
]

print("=" * 70)
print("LONG HR CONSULTATION SESSION")
print("=" * 70)

for i, message in enumerate(consultation_messages, 1):
    print(f"\n{'='*70}")
    print(f"Turn {i}/10")
    print(f"{'='*70}")
    print(f"👤 Priya: {message}")
    
    result = hr_consultation_agent.invoke(
        {"messages": [{"role": "user", "content": message}]},
        config
    )
    
    response = result['messages'][-1].content
    print(f"\n🤖 HR Agent: {response[:200]}...")
    print(f"\nTotal messages in state: {len(result['messages'])}")

print("\n" + "=" * 70)
print("✅ Long conversation handled with automatic summarization!")
print(f"\n💡 Summary created: {summarization_mw.summary if summarization_mw.summary else 'Not yet needed'}")

---
# Part 2: HumanInTheLoopMiddleware 👤

## Why Human-in-the-Loop?

**Critical HR Operations Need Approval:**
- 💰 Salary updates
- 🗑️ Employee termination
- 📝 Contract changes
- 🎯 Performance reviews
- 🏆 Promotions

**HumanInTheLoopMiddleware Features:**
- Intercepts tool calls before execution
- Requests human approval
- Allows editing tool arguments
- Supports rejection with explanation
- Provides manual override option

---

## Lab 2.1: Basic Human-in-the-Loop Setup

In [ ]:
# Simplified Human-in-the-Loop Middleware
class SimpleHumanInTheLoopMiddleware:
    """Human approval for sensitive operations."""
    
    def __init__(self, tools_requiring_approval: list[str]):
        self.approval_required = tools_requiring_approval
        self.pending_approvals = {}
        self.approval_history = []
    
    def post_model_hook(self, state: AgentState) -> dict:
        """Intercept tool calls and request approval."""
        messages = state.get("messages", [])
        
        # Check for tool calls
        for msg in messages:
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                for tool_call in msg.tool_calls:
                    tool_name = tool_call.get('name', '')
                    
                    if tool_name in self.approval_required:
                        approval_id = f"approval_{len(self.pending_approvals) + 1}"
                        
                        # Store pending approval
                        approval_request = {
                            "id": approval_id,
                            "tool": tool_name,
                            "args": tool_call.get('args', {}),
                            "status": "pending",
                            "timestamp": datetime.now().isoformat(),
                            "requested_by": state.get("current_user_id", "unknown")
                        }
                        
                        self.pending_approvals[approval_id] = approval_request
                        
                        print(f"\n✋ [APPROVAL REQUIRED]")
                        print(f"   ID: {approval_id}")
                        print(f"   Tool: {tool_name}")
                        print(f"   Arguments: {tool_call.get('args', {})}")
                        print(f"   Requested by: {approval_request['requested_by']}")
                        
                        # Pause execution
                        return {
                            "messages": [(
                                "assistant",
                                f"⏸️  **Approval Required**\n\n"
                                f"Operation: `{tool_name}`\n"
                                f"Details: {json.dumps(tool_call.get('args', {}), indent=2)}\n\n"
                                f"Approval ID: `{approval_id}`\n\n"
                                f"This operation requires manager approval. "
                                f"Please have your manager review and approve this request."
                            )],
                            "jump_to": "__end__"  # Stop execution
                        }
        
        return {}
    
    def approve(self, approval_id: str, approved: bool, approver_id: str = "unknown", notes: str = ""):
        """Approve or reject an operation."""
        if approval_id in self.pending_approvals:
            approval = self.pending_approvals[approval_id]
            approval["status"] = "approved" if approved else "rejected"
            approval["approver_id"] = approver_id
            approval["approval_timestamp"] = datetime.now().isoformat()
            approval["notes"] = notes
            
            # Move to history
            self.approval_history.append(approval)
            del self.pending_approvals[approval_id]
            
            return True
        return False
    
    def edit_and_approve(self, approval_id: str, new_args: dict, approver_id: str = "unknown"):
        """Edit arguments and approve."""
        if approval_id in self.pending_approvals:
            approval = self.pending_approvals[approval_id]
            approval["original_args"] = approval["args"].copy()
            approval["args"] = new_args
            approval["status"] = "approved_with_edits"
            approval["approver_id"] = approver_id
            approval["approval_timestamp"] = datetime.now().isoformat()
            
            self.approval_history.append(approval)
            del self.pending_approvals[approval_id]
            
            return True
        return False
    
    def get_pending_approvals(self):
        """Get all pending approvals."""
        return self.pending_approvals
    
    def get_approval_history(self):
        """Get approval history."""
        return self.approval_history

print("✅ SimpleHumanInTheLoopMiddleware created!")

## Lab 2.2: HR Operations with Approval Workflow

In [ ]:
# Create human-in-the-loop middleware
hitl_mw = SimpleHumanInTheLoopMiddleware(
    tools_requiring_approval=["update_salary", "approve_leave"]
)

# Custom state for tracking user info
class HRAgentState(AgentState):
    current_user_id: str = ""
    current_user_role: str = ""

# Create agent with approval workflow
hr_approval_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, update_salary, approve_leave],
    post_model_hook=hitl_mw.post_model_hook,
    state_schema=HRAgentState,
    prompt="""You are an HR operations assistant.
    
    You help with:
    - Employee information lookup
    - Leave balance checks
    - Salary updates (requires approval)
    - Leave approvals (requires manager approval)
    
    Always be professional and follow proper procedures."""
)

print("✅ HR Agent with Approval Workflow ready!")

## Lab 2.3: Test Approval Workflow - Salary Update

In [ ]:
print("=" * 70)
print("SCENARIO 1: Salary Update Request")
print("=" * 70)

# Step 1: Request salary update
print("\n📝 Step 1: Employee requests salary update...\n")

result = hr_approval_agent.invoke({
    "messages": [{"role": "user", "content": "Please update the salary for employee 101 (Priya Sharma) to ₹150,000"}],
    "current_user_id": "102",
    "current_user_role": "Engineering Manager"
})

print(f"🤖 Agent Response:\n{result['messages'][-1].content}")

# Step 2: Check pending approvals
print("\n" + "=" * 70)
print("📋 Step 2: Review pending approvals...")
print("=" * 70)

pending = hitl_mw.get_pending_approvals()
print(f"\nPending approvals: {len(pending)}")

for approval_id, details in pending.items():
    print(f"\n🔍 Approval Details:")
    print(json.dumps(details, indent=2))
    
    # Step 3: Manager reviews and approves
    print("\n" + "=" * 70)
    print("✅ Step 3: Manager approves...")
    print("=" * 70)
    
    hitl_mw.approve(
        approval_id=approval_id,
        approved=True,
        approver_id="103",  # HR Director Anjali Patel
        notes="Approved based on performance review and market adjustment"
    )
    print(f"✅ Approval {approval_id} APPROVED by HR Director (103)")

# Step 4: View approval history
print("\n" + "=" * 70)
print("📜 Step 4: Approval History")
print("=" * 70)

history = hitl_mw.get_approval_history()
for entry in history:
    print(f"\n{entry['id']}:")
    print(f"  Tool: {entry['tool']}")
    print(f"  Status: {entry['status']}")
    print(f"  Requested by: {entry['requested_by']}")
    print(f"  Approved by: {entry.get('approver_id', 'N/A')}")
    print(f"  Notes: {entry.get('notes', 'N/A')}")

print("\n✅ Approval workflow completed!")

## Lab 2.4: Test Rejection Scenario

In [ ]:
print("=" * 70)
print("SCENARIO 2: Leave Approval Request - REJECTION")
print("=" * 70)

# Request leave approval
print("\n📝 Step 1: Request leave approval...\n")

result = hr_approval_agent.invoke({
    "messages": [{"role": "user", "content": "Approve 20 days leave for employee 101"}],
    "current_user_id": "102",
    "current_user_role": "Engineering Manager"
})

print(f"🤖 Agent Response:\n{result['messages'][-1].content}")

# Check and reject
pending = hitl_mw.get_pending_approvals()

for approval_id, details in pending.items():
    print(f"\n🔍 Approval Request:")
    print(json.dumps(details, indent=2))
    
    print("\n" + "=" * 70)
    print("❌ Step 2: Manager REJECTS...")
    print("=" * 70)
    
    hitl_mw.approve(
        approval_id=approval_id,
        approved=False,
        approver_id="102",  # Engineering Manager Rahul Verma
        notes="Rejected: Critical project deadline. Please reschedule for next month."
    )
    print(f"❌ Approval {approval_id} REJECTED by Manager (102)")

# View updated history
print("\n" + "=" * 70)
print("📜 Complete Approval History")
print("=" * 70)

for entry in hitl_mw.get_approval_history():
    status_icon = "✅" if entry['status'] == "approved" else "❌"
    print(f"\n{status_icon} {entry['id']} - {entry['status'].upper()}")
    print(f"   Tool: {entry['tool']}")
    print(f"   Args: {entry['args']}")
    print(f"   Requested: {entry['timestamp']}")
    print(f"   By: {entry['requested_by']}")
    print(f"   Approver: {entry.get('approver_id', 'N/A')}")
    print(f"   Notes: {entry.get('notes', 'N/A')}")

## Lab 2.5: Edit-and-Approve Scenario

In [ ]:
print("=" * 70)
print("SCENARIO 3: Salary Update with EDITS")
print("=" * 70)

# Request unrealistic salary increase
print("\n📝 Step 1: Request salary update...\n")

result = hr_approval_agent.invoke({
    "messages": [{"role": "user", "content": "Update salary for employee 104 (Arjun Reddy) to ₹300,000"}],
    "current_user_id": "105",
    "current_user_role": "Sales Director"
})

print(f"🤖 Agent Response:\n{result['messages'][-1].content}")

# Manager edits and approves with lower amount
pending = hitl_mw.get_pending_approvals()

for approval_id, details in pending.items():
    print(f"\n🔍 Original Request:")
    print(f"   Employee: {details['args']['employee_id']}")
    print(f"   Requested Salary: ₹{details['args']['new_salary']:,}")
    
    print("\n" + "=" * 70)
    print("✏️  Step 2: Manager EDITS and APPROVES...")
    print("=" * 70)
    
    # Edit to more reasonable amount
    new_args = {
        "employee_id": details['args']['employee_id'],
        "new_salary": 180000  # Edited from 300000 to 180000
    }
    
    hitl_mw.edit_and_approve(
        approval_id=approval_id,
        new_args=new_args,
        approver_id="103"  # HR Director
    )
    
    print(f"✅ Approval {approval_id} EDITED and APPROVED")
    print(f"   Original: ₹{details['args']['new_salary']:,}")
    print(f"   Approved: ₹{new_args['new_salary']:,}")
    print(f"   Reason: Adjusted to market rate and budget constraints")

# Final history
print("\n" + "=" * 70)
print("📊 COMPLETE APPROVAL AUDIT TRAIL")
print("=" * 70)

for i, entry in enumerate(hitl_mw.get_approval_history(), 1):
    print(f"\n{'='*70}")
    print(f"Approval #{i}: {entry['id']}")
    print(f"{'='*70}")
    print(f"Operation: {entry['tool']}")
    print(f"Status: {entry['status'].upper()}")
    print(f"Requested by: {entry['requested_by']} at {entry['timestamp']}")
    print(f"Decided by: {entry.get('approver_id', 'N/A')} at {entry.get('approval_timestamp', 'N/A')}")
    
    if 'original_args' in entry:
        print(f"\nOriginal Arguments:")
        print(f"  {json.dumps(entry['original_args'], indent=2)}")
        print(f"Modified Arguments:")
        print(f"  {json.dumps(entry['args'], indent=2)}")
    else:
        print(f"Arguments: {json.dumps(entry['args'], indent=2)}")
    
    if entry.get('notes'):
        print(f"Notes: {entry['notes']}")

print("\n✅ All approval scenarios demonstrated!")

---
# Part 3: Combining Both Middlewares

**Production Pattern:** Use both summarization and approval together

In [ ]:
# Create combined middleware
combined_summarization_mw = SimpleSummarizationMiddleware(
    model=llm,
    max_tokens=1000,
    messages_to_keep=5
)

combined_hitl_mw = SimpleHumanInTheLoopMiddleware(
    tools_requiring_approval=["update_salary", "approve_leave"]
)

# Combine hooks
def combined_pre_hook(state: AgentState) -> dict:
    """Run summarization check."""
    return combined_summarization_mw.pre_model_hook(state)

def combined_post_hook(state: AgentState) -> dict:
    """Run approval check."""
    return combined_hitl_mw.post_model_hook(state)

# Create production-ready agent
production_hr_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, get_salary_info, update_salary, approve_leave],
    pre_model_hook=combined_pre_hook,
    post_model_hook=combined_post_hook,
    state_schema=HRAgentState,
    checkpointer=InMemorySaver(),
    prompt="""You are a comprehensive HR assistant.
    
    Capabilities:
    - Employee information and queries
    - Leave management
    - Salary operations (requires approval)
    - Long consultation sessions (with auto-summarization)
    
    You maintain context across long conversations and ensure
    all critical operations get proper approval."""
)

print("✅ Production HR Agent with BOTH middlewares ready!")
print("\nFeatures:")
print("  📝 Auto-summarization for long conversations")
print("  👤 Human approval for critical operations")
print("  💾 Conversation persistence")
print("  🔒 Security and compliance")

---
# Summary & Best Practices

## Built-in Middleware Overview

| Middleware | Purpose | When to Use |
|------------|---------|-------------|
| **SummarizationMiddleware** | Auto-summarize old messages | Long conversations, consulting sessions |
| **HumanInTheLoopMiddleware** | Require approval for tools | Critical operations, compliance needs |

## Key Patterns Learned

### 1. Summarization Pattern
```python
summarization_mw = SimpleSummarizationMiddleware(
    model=llm,
    max_tokens=2000,
    messages_to_keep=5
)
agent = create_agent(
    ...,
    pre_model_hook=summarization_mw.pre_model_hook
)
```

### 2. Approval Pattern
```python
hitl_mw = SimpleHumanInTheLoopMiddleware(
    tools_requiring_approval=["sensitive_tool"]
)
agent = create_agent(
    ...,
    post_model_hook=hitl_mw.post_model_hook
)

# Later: approve/reject
hitl_mw.approve(approval_id, approved=True, approver_id="manager")
```

## Production Checklist

✅ **Summarization:**
- Set appropriate token thresholds
- Keep enough recent messages for context
- Test summary quality
- Monitor token usage

✅ **Human-in-the-Loop:**
- Identify tools requiring approval
- Implement notification system
- Store approval history for audit
- Set timeout policies
- Handle approval UI/UX

✅ **Combined:**
- Use both for comprehensive HR agents
- Chain hooks properly
- Test interaction between middlewares
- Document approval workflows

## Next Steps

- Explore decorator-based middleware
- Learn class-based middleware patterns
- Implement custom middleware
- Build production approval systems

---

**Congratulations!** You now know how to use built-in middlewares for production HR agents! 🎉